In [1]:
# Dependencies
import requests
import json
import gmaps
import pandas as pd
import numpy as np
from config import gkey
from pprint import pprint

In [2]:
# read csv file
file="output_data/retrieved_city_data.csv"
cities_data_df=pd.read_csv(file)
city_df=(cities_data_df.drop(['Unnamed: 0','Lng','Lat'],axis=1)).dropna()

city_df.head()

,City,Country,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph)
0,punta arenas,cl,51.80,81.0,40.0,10.29,23.018113
1,khategaon,in,75.13,29.0,45.0,4.70,10.513618
4,lorengau,pg,84.52,70.0,100.0,7.58,16.956005
6,carnarvon,au,80.60,69.0,100.0,16.11,36.037103
7,port alfred,za,75.61,77.0,83.0,27.42,61.336895


In [3]:
#get lat lng for cities
city_df['Lat']=""
city_df["Lng"]=""

# use iterrows to iterate through pandas dataframe
for index, row in cities_data_df.iterrows():
    
    target_city=row["City"]
    region=row["Country"]
    
    # target url
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&region={1}&key={2}').format(target_city,region, gkey)
    geo_data = requests.get(target_url).json()
    results = geo_data['results']
    # Extract latitude and longitude
    try:
       
        city_df.loc[index, 'Lat']= results[0]["geometry"]["location"]["lat"]
        city_df.loc[index, 'Lng']= results[0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        
        
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
city_df.head(200)



city_df.dtypes


City                 object
Country              object
Max Temp (F)        float64
Humidity (%)        float64
Cloudiness (%)      float64
Wind Speed (m/s)    float64
Wind Speed (mph)    float64
Lat                 float64
Lng                 float64
dtype: object

In [7]:
gmaps.configure(api_key=gkey)


# Convert humidity to float, store and  handle NaN values

# Store 'Lat' and 'Lng' into  locations 
city_df["Lat"] = pd.to_numeric(city_df["Lat"] , errors='coerce')
city_df["Lng"] = pd.to_numeric(city_df["Lng"] , errors='coerce')
city_df = city_df.dropna()
locations=city_df[["Lat","Lng"]]
humidity = city_df["Humidity (%)"].astype(float)

In [9]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
#define a ideal location for vacation
ideal_df= city_df.loc[( city_df["Max Temp (F)"] <80.0) & (city_df["Max Temp (F)"]>50.0) ]

ideal_df=ideal_df.loc[(ideal_df["Wind Speed (mph)"]<10) & (ideal_df["Cloudiness (%)"]==0) ]


# Convert humidity to float and store
# handle NaN values
ideal_df = ideal_df.dropna()
# Store 'Lat' and 'Lng' into  locations 
ideal_locations = ideal_df[["Lat", "Lng"]].astype(float)
ideal_humidity = ideal_df["Humidity (%)"].astype(float)
ideal_df.head()

ideal_df['Hotel Name'] = ""
ideal_df['Address'] = ""
ideal_df.head(18)

,City,Country,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph),Lat,Lng,Hotel Name,Address
76,sagua de tanamo,cu,79.63,57.0,0.0,3.15,7.046361,20.580036,-75.243531,,
245,kyaikto,mm,72.79,52.0,0.0,3.18,7.113469,17.312468,97.014901,,
362,jishou,cn,54.95,87.0,0.0,1.54,3.444888,28.262375,109.698015,,
400,kununurra,au,78.80,78.0,0.0,1.12,2.505373,-15.604949,128.765468,,
446,makhu,in,57.87,56.0,0.0,3.47,7.762182,31.107153,74.976402,,
519,ayotzintepec,mx,72.41,57.0,0.0,2.15,4.809421,17.672837,-96.128773,,
530,qui nhon,vn,70.86,94.0,0.0,0.83,1.856660,13.782967,109.219663,,
560,ca mau,vn,76.50,90.0,0.0,4.27,9.551734,9.152673,105.196079,,


In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# find the closest hotel of each type to coordinates


target_radius = 5000
target_type = "lodging"
# set up a parameters dictionary
params = {
    
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [18]:
# use iterrows to iterate through pandas dataframe
for index, row in  ideal_df.iterrows():

    # add keyword to params dict
    params['location'] =f"{row['Lat']},{row['Lng']}"

    # assemble url and make API request
    #print(f" Retrieving lodging for Index {index} : {row['Lat']}, {row['Lng']}.")
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    # extract results
    results = response['results']
    
    try:
        print(f"first {row['Lat']},{row['Lng']} Hotal is {results[0]['name']}.")
        
        ideal_df.loc[index, 'Hotel Name'] = results[0]['name']
        ideal_df.loc[index, 'Address'] = results[0]['vicinity']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
------------
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAALQbk2Udcz6R_B54IeCC8AlWe6Sa41bAdKXb4Z9lS9pKCDxJP_x5E-DJEZ0KIWYIN2vNo5LJ18FNDt9A3oigGh6yQlwVIFyv-Weagn_NFVv-WEUsz54JUzzZtxm8M2mxJyrNOXIxl-luBB2OntGD13MRMIQDRddFYEasSkxlf3IBCSlwRjSAvDB6DfYI_Ni_VPmnghIH2eU9zF4bOM7ZOw6c1SolOh2qqlhBV1F8zlVt0sWxCCVf96sIzq6KJFyqUJtecCHz8hrwBGD8ofdQsUqacIvO7njFFO-vsjc5Cb5SKk_rLkUAdbuMdYrhoU1mOF54Qd-syc2OF1P_I_-QmXmLWM8wx8cuofWQRjSd0KskivVMvmvSceOBfDb-zF3axRIQqoFgLH7MM3LZCJGmfRQ_QRoUO-QXsGdGsy7tjvDyq0grMX8YYHM",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 17.3119714,
                    "lng": 97.01586019999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.3132529302915,
                        "lng": 97.0170819802915
                   

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAALl077iK-2N3zxJ7uUqzRtBAszDR9MD1ycir2nJvN-WNna0U8bxu7e8cCbH0SNXqHNYNHJj_IXQ-r_DXiyJ06-CXOp0rOr0xnH1UwZdOX5S7eFCwfJEtzjpcbFCgEM2Cl7YDpvRlzv_2937jECdjgvK6sMVv4W3xw_L5h9r0lWQVA8sn0IkEDDgS7a6meAKfupwj48UrIAKNXa5PzuqaI1ZmkWbUAQZUrRFwZISL7LfKE3Kh5sNDa5JA2ZXa38UFugs5itUOIYuByniWamDfAybxfiD58KZMoovgx2diAkpBPhdh3ZGoF_U6TGPUujXexIOXJnP-KeSWOkocx8RkLdiS2ekN_BY5XdFxXcrMBDRFbKnBh0wcWdltKTQaiYPE7hIQdkShgI8HUyHPAidTuAqMTBoUgXD2OTJGE6OYvcmwIKXlUk0oYoA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 28.265145,
                    "lng": 109.706648
                },
                "viewport": {
                    "northeast": {
                        "lat": 28.2664200802915,
                        "lng": 109.7079409802915
                    },
                    "southwest": {
                        "lat": 28.2637221197085,
                        "lng": 109.705243019708

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 31.1071528,
                    "lng": 74.9764024
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.1085274302915,
                        "lng": 74.97801018029149
                    },
                    "southwest": {
                        "lat": 31.1058294697085,
                        "lng": 74.97531221970848
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "28cf8ad678399a846ab1f09207bd370e7bb9664e",
            "name": "Deep glass art",
            "place_id": "ChIJ85CEJDshGjkRscx5bmUVbhc",
            "plus_code": {
                "compound_code": "4X4G+VH Makhu, Punjab, India",
                "global_code": "8J3P4X4G+VH"
            },
            "referenc

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 17.6745136,
                    "lng": -96.1280852
                },
                "viewport": {
                    "northeast": {
                        "lat": 17.67584753029151,
                        "lng": -96.1267002197085
                    },
                    "southwest": {
                        "lat": 17.6731495697085,
                        "lng": -96.1293981802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "49ef6443cb280365be86281d74f0e7053a4b4441",
            "name": "Hotel \"Punta del Cerro\"",
            "place_id": "ChIJ8cSQchZDwYURWDBhbuK8hcc",
            "plus_code": {
                "compound_code": "MVFC+RQ Ayotzintepec, Oaxaca, Mexico",
                "global_code": "7695MVFC+RQ"
            },


{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAANwfeuBAfQ_x7w2hFj42UOuXrZ3VAHeLSdSiLVjSgJwClt3_-qIjHNZOPhXnxxOrbR5TuehLqxWW3uOKBo-60GxPREYIFoNLIKa8B8kkMxK1nSwVab5jWh7yCIeQ62MhqckqKDBrEFV9-IlL6g3lJyNXuRuR0r3JobtWPizc4BYIfxlH891w1FUIylZnbu8tp-ctaM8JpQNwyeMR9qK_9Dy_PpJbrmhw-Ajv_CEzgQYLExM3R4HFkb3CfTOonAoGovcy8oHR-jwDJof0qw803ct9q-E4pTrdtbkHmzttB8g_IHzd-Wkle2HXH2Nf7baM5P4r2HcfbYtdy5MynMj7USJs6mCzLgdNXZLFlqOvmMVXPmHzCQLvu0nlM30M_cEO7hIQgZL9_-OCj6xNz3V5nY6bjBoUsAPiKFbnsSleMS-VEEIMbUFsEUc",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 9.177367599999998,
                    "lng": 105.1620099
                },
                "viewport": {
                    "northeast": {
                        "lat": 9.178760930291503,
                        "lng": 105.1633572802915
                    },
                    "southwest": {
                        "lat": 9.176062969708498,
                        "lng": 105.1

In [20]:
 ideal_df.dropna()

,City,Country,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (m/s),Wind Speed (mph),Lat,Lng,Hotel Name,Address
76,sagua de tanamo,cu,79.63,57.0,0.0,3.15,7.046361,20.580036,-75.243531,,
245,kyaikto,mm,72.79,52.0,0.0,3.18,7.113469,17.312468,97.014901,Moe,Kyaikto
362,jishou,cn,54.95,87.0,0.0,1.54,3.444888,28.262375,109.698015,Xiangzhou International Hotel,"乾州溶江小区, Jishou, 湘西土家族苗族自治州吉首市"
400,kununurra,au,78.80,78.0,0.0,1.12,2.505373,-15.604949,128.765468,,
446,makhu,in,57.87,56.0,0.0,3.47,7.762182,31.107153,74.976402,Deep glass art,Makhu
519,ayotzintepec,mx,72.41,57.0,0.0,2.15,4.809421,17.672837,-96.128773,"Hotel ""Punta del Cerro""",Ayotzintepec
530,qui nhon,vn,70.86,94.0,0.0,0.83,1.856660,13.782967,109.219663,Seagull Hotel,"489 An Dương Vương, Nguyễn Văn Cừ, Thành phố Q..."
560,ca mau,vn,76.50,90.0,0.0,4.27,9.551734,9.152673,105.196079,Nguyen Tuan Guesthouse,"29 Trần Văn Thời, Phường 6, Thành phố Cà Mau"


In [36]:
# Store 'Lat' and 'Lng' into  locations 
ideal_locations = ideal_df[["Lat", "Lng"]].astype(float)
ideal_humidity = ideal_df["Humidity (%)"].astype(float)
ideal_df.head()

hotel_df=ideal_df.loc[(ideal_df["Hotel Name"]!='')][["Hotel Name","City","Country","Lat", "Lng"]]


pandas.core.frame.DataFrame

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(ideal_locations, weights=ideal_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)


markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [240]:
hotel_df

,name,City,Country,location
37,محطة تاكسيات الشاطئ براك,sabha,ly,"27.0365406,14.4290236"
194,Fishermans Guest House,henties bay,na,"-22.1134964,14.2832038"
220,Pousada Barra Sul,laguna,br,"-28.4876062,-48.8397403"
265,Qom International Hotel,qom,ir,"34.6415764,50.8746035"
272,Gran Tacande Wellness & Relax Costa Adeje,adeje,es,"28.1220604,-16.7338211"
312,Dr. Mohsen Villa,jalu,ly,"29.040621,21.4991334"
501,Beachfront Hotel Hokitika,hokitika,nz,"-42.7173686,170.9665427"
